#Support Vector Machine for Credit Data

In [4]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.cross_validation import cross_val_score, ShuffleSplit
import sklearn as sk

In [8]:
from scipy.stats import sem

def mean_score(scores):
    return ('Mean score: {0:.3f} (+/-{1:.3f})').format(np.mean(scores), sem(scores))

In [9]:
df = pd.read_csv('data/cs-train_clean.csv')
X = df.drop('SeriousDlqin2yrs', axis=1)
y = df.SeriousDlqin2yrs

In [16]:
svc = SVC(kernel='rbf', C=1, gamma=0.001)
cv_svm = ShuffleSplit(10000, n_iter=10, test_size=0.25, random_state=0)

In [17]:
test_scores_svm = cross_val_score(svc, X, y, cv=cv_svm, n_jobs=2)
test_scores_svm
print mean_score(test_scores_svm)

Mean score: 0.937 (+/-0.001)


while tweaking the parameters for svm, I noticed an increase